In [27]:
from pyomo.environ import *

In [30]:
#data
connections = {
    (1,2):1,
    (1,4):1,
    (2,1):1,
    (2,3):1,
    (2,5):1,
    (3,2):1,
    (3,5):1,
    (3,6):1,
    (4,1):1,
    (4,5):1,
    (5,2):1,
    (5,3):1,
    (5,4):1,
    (5,6):1,
    (6,3):1,
    (6,5):1,
}   

cost = {1:40,2:65,3:43,4:48,5:72,6:36}

In [31]:
#Model
m = ConcreteModel()

#define sets
m.nodes = Set(initialize = range(1, 7))
adj = list(connections.keys()) #converting keys of connections into a list
m.A = Set(within = m.nodes*m.nodes, initialize=adj) #defining adjacency matrix

#Parameters
m.connections = Param(m.A, initialize=connections)
m.cost = Param(m.nodes, initialize=cost)

#Decision variables
m.x = Var(m.nodes, domain=Binary)

#Constraint
def min_node_cover(m, i, j):
    return (m.x[i] + m.x[j]) >= m.connections[i,j] 
m.ProjectConstraints = Constraint(m.A, rule=min_node_cover)

#Objective function
m.obj = Objective(expr = sum(m.cost[i]*m.x[i] for i in m.nodes), sense=minimize)

#Solve the model
SolverFactory('gurobi').solve(m)

#print result
print("Selected nodes are:")
for i in nodes:
    if (m.x[i].value == 1):
        print(f"Node {i}")
print("Minimum total cost is:",m.obj.expr())

Selected nodes are:
Node 1
Node 3
Node 5
Minimum total cost is: 155.0
